# Single Subject Segmentation Q1K - Resting state

In [ ]:
# #use these commented parameters for testing this notebook outside of the automated loop of q1k_automated_reports.ipynb
# subject_id = "0042P"
# task_id = "RS"
# session_id = "01"
# run_id = "1"

#use these parameters when executing this notebook from the automation notebook..
subject_id = ""
task_id = ""
session_id = ""
run_id = ""


In [ ]:
# import packages
import mne
import mne_bids
import numpy as np
import pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import shutil
import pylossless as ll

import warnings
warnings.filterwarnings('ignore')


In [ ]:
common_freqs = {
    'delta' : (0,4),
    'theta': (4, 7),
    'alpha': (8, 12),
    'beta':(13, 30),
    'gamma': (30, 45)
}

frontal_roi = ['E18','E19','E23','E24','E27','E3','E4','E10','E118','E123','E124']

# Set the parameters and read the pylossless data.

In [ ]:
project_path = "/project/def-emayada/q1k/experimental/"
pylossless_path = "derivatives/pylossless/"
sync_loss_path = "derivatives/sync_loss/"
segment_path = "derivatives/segment/"

bids_path = mne_bids.BIDSPath(
    subject=subject_id, session=session_id, task=task_id, run="1", datatype="eeg", suffix="eeg",root=project_path + pylossless_path + sync_loss_path
)
print(bids_path)

In [ ]:
# Read the BIDS pylossless output file..
eeg_raw = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False)

In [ ]:
eeg_raw.info

In [ ]:
eeg_events, eeg_event_dict  = mne.events_from_annotations(eeg_raw)
eeg_event_dict

In [ ]:
eeg_raw.annotations.description

## Plot events

In [ ]:
fig=mne.viz.plot_events(
    eeg_events, sfreq=eeg_raw.info["sfreq"], first_samp=eeg_raw.first_samp, 
    event_id=eeg_event_dict
)

### Define all rest events

In [ ]:
annot_sample = []
annot_id = []
freq = eeg_raw.info["sfreq"]

# Select all annotations 
annots = [a for a in eeg_raw.annotations if a["description"] in ["dbrk", "vs01", "vs02","vs03","vs04", "vs05","vs06"]]

# Add an end annotation - there are 60 seconds after the last annotation
annots.append(OrderedDict((("onset", annots[-1]["onset"] + 60.), ("duration", 0),
                               ("description", "end"), ('orig_time', None))))

# Create new annotations for epochs
for annot, next_annot in zip(annots[:-1], annots[1:]):
    if annot["description"] == "dbrk" or annot["description"]=="end":
        continue

    annot_sample.append(np.arange(int(annot["onset"] * freq),
                                    int(next_annot["onset"] * freq),
                                    int(1 * freq)))
    id_ = eeg_event_dict[annot["description"]]
    annot_id.extend(id_ * np.ones(len(annot_sample[-1])))

annot_sample = np.concatenate(annot_sample)

rs_eeg_events=np.array([annot_sample, [0] * len(annot_sample), annot_id], dtype=int).T
rs_eeg_events = mne.merge_events(rs_eeg_events, np.unique(rs_eeg_events[:, -1]), 1)


In [ ]:
# Check events after creation of event structure
fig=mne.viz.plot_events(
    rs_eeg_events, sfreq=eeg_raw.info["sfreq"], first_samp=eeg_raw.first_samp,
)

In [ ]:
channel_types = eeg_raw.get_channel_types()
print("Channel Types:", channel_types)
print("Channel Names:", eeg_raw.info['ch_names'])

# Interpret the annotations and apply the data selections.

In [ ]:
#show channel types...
channel_types = eeg_raw.get_channel_types()
print("Channel Types:", channel_types)
print("Channel Names:", eeg_raw.info['ch_names'])

In [ ]:
rs_event_id= {"rest":1}

# Segment the data to create epochs

In [ ]:
# Epoch structure is created for ALL events, then you 'segment' by indexing into it
epochs = mne.Epochs(eeg_raw, rs_eeg_events, tmin=-0.2, tmax=0.8, on_missing='warn', event_id=rs_event_id)

epochs.save(project_path + pylossless_path + sync_loss_path + segment_path + 'epoch_fif_files/RS/' + bids_path.basename + '_epo.fif', overwrite=True)

In [ ]:
epochs = epochs[['rest']]
display(epochs)
#epochs.save(project_path + pylossless_path + postproc_path + 'epoch_fif_files/PLR/' + bids_path.basename + '_epo.fif', overwrite=True)

In [ ]:
epo_power= epochs.compute_psd(fmin=0,fmax=50)
psds, freqs = epo_power.get_data(return_freqs=True)
print(f"\nPSDs shape: {psds.shape}, freqs shape: {freqs.shape}")

## Plot power spectrum

In [ ]:
fig=epo_power.plot(picks="eeg", exclude="bads", amplitude=False)

In [ ]:
fig=epo_power.plot(picks=frontal_roi, exclude="bads", amplitude=False)

In [ ]:
fig=epo_power.plot_topomap(ch_type="eeg", agg_fun=np.median)

In [ ]:
#!jupyter nbconvert --output {"session_reports/RS/" + bids_path.basename + ".html"} --TagRemovePreprocessor.remove_all_outputs_tags='{"exclude"}' --no-input --to html session_seg_RS.ipynb